# Power diffraction action

This notebook is to demonstrate the process of extracting structure from powerder diffraction data

Version 1.0, updated 07/10/2021 by AJM

## Import packages

To streamline operations in Python, packages can be imported to perform a host of various tasks. To make this process as simple as possible, all the required packages are included in the file _[SSP.py](https://github.com/Andy-UTAS/Solid-state/blob/master/SSP.py)_ and thus we can import all of the content: 

In [ ]:
from SSP import *

## Import the data

In [ ]:
# Import the data, noting the very nonstandard .x_y file format, which requires read engine specification
data = pd.read_csv('neutronscatter.x_y', delim_whitespace=True, engine = 'python', names=['Angle', 'Intensity'])
data['Intensity'] = data['Intensity']/data['Intensity'].max() # Normalise the intensity

# Plot the data

fig, ax = plt.subplots()

ax.plot(data['Angle'], data['Intensity'], label = r'$mv^2/\beta$: 7.47 keV', color = 'C0')
ax.set_xlabel(r'Angle [2$\theta$]')
ax.set_ylabel('Intensity [arb]')
ax.set_title('X-ray scattering of unknown material');
plt.legend()

ax.set_xlim((19,82))

loc = matplotlib.ticker.MultipleLocator(1)
ax.xaxis.set_minor_locator(loc)
ax.tick_params(which='minor')

ax.set_aspect(30) # if adjusting the aspect ratio, make sure to use bbox_inches='tight' when saving!

if False:
    plt.savefig('neutronscatter.pdf', facecolor='white', transparent=False, bbox_inches='tight')

plt.show()

## Find the location of the peaks

In [ ]:
'''
The function find_peaks is amazing, but sometimes needs a bit of assistance to function well. 
For example, in order to avoid the detection of some spurious peaks, we set a minimum peak prominence of 0.01
With prominence of zero (the default), you will see some rubbish peaks

The functionality is very useful when working with noisy data!
'''

peaks, _ = find_peaks(data['Intensity'], prominence = 0.01) # Find the peaks

# Plot the data with peaks

fig, ax = plt.subplots()

ax.plot(data['Angle'], data['Intensity'], label = r'$mv^2/\beta$: 7.47 keV', color = 'C0')
ax.plot(data['Angle'][peaks], data['Intensity'][peaks], "x", color = 'C1', label = 'Peaks')
ax.set_xlabel(r'Angle [2$\theta$]')
ax.set_xlim((19,82))
ax.set_ylabel('Intensity [arb]')
ax.set_title('X-ray scattering of unknown material');
plt.legend()

plt.show()

## Extract the information of interest

In [ ]:
# Collect the peak data
analysis = pd.DataFrame() # Make a dataframe for the results
analysis['2theta'] = data['Angle'][peaks] # Add the diffraction angles

# Aesthetics
analysis.reset_index(drop = True, inplace = True) # Clean the index - just to make things pretty
analysis.index += 1 # Make index the peak number

# Experimental parameters
momenergy = 7477.72 # momenergy - mv^2/\beta of neutrons [eV]
wl = const.h * const.c / (momenergy * e) # Convert energy to wavelength [m]

# Perform calculations
analysis['d'] = wl / (2 * np.sin(np.deg2rad(analysis['2theta']/2))) # Calculate the plane spacing from Bragg condition [m]
d_a = analysis['d'].iloc[0] # First diffraction peak (d_a)
analysis['ratio'] = (d_a/analysis['d']) ** 2 # Ratio of angles of first peak to subsequent peaks

# Show the data
analysis.head(10) # Note the functionality of head(x) shows the first x rows of a dataframe

## Find the Miller indices

In [ ]:
# Factor by which we multiply the ratio to obtain N = h^2 + k^2 + l^2 (which is an integer)
guess = 4

analysis['N guess'] = (guess * analysis['ratio']) # Evaluate guess * ratio
analysis['N int'] = analysis['N guess'].round(0).astype(int) # Get the integer version of the above result
analysis['N error'] = 100 * (analysis['N guess'] - analysis['N int'])/analysis['N guess'] # Calculate the percentage error between the integer version and the ratio * guess

# Show the data
analysis